In [23]:
# Step 1: Import necessary libraries
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from utils.classification_data import linear_data, checkerboard_data, power_line_data, microgrid_data, make_double_cake_data
from sklearn.model_selection import train_test_split
from pennylane import numpy as np
import pennylane as qml
from pennylane import numpy as np
import matplotlib as mpl
from pennylane import numpy as np
from sklearn.cluster import SpectralClustering, KMeans
from collections import deque
import pandas as pd

#data = checkerboard_data(2)

## Extract features and target
#features = np.asarray(data.drop(columns=['target']))
#target = np.asarray(data['target'])
#target = target % 2
#target = 2 * target - 1

#X, x_test, Y, y_test = train_test_split(
#    features, target, test_size=0.2, random_state=42
#)

In [24]:
data = pd.read_csv('data/balanced_quantum_dataset.csv')
features = np.asarray(data.drop(columns=['label']))
target = np.asarray(data['label'])

X, x_test, Y, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

In [25]:

np.random.seed(1359)
circuit_executions = 0
def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])

def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))


adjoint_ansatz = qml.adjoint(ansatz)


def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

dev = qml.device("default.qubit", wires=5, shots=None)
wires = dev.wires.tolist()

@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    global circuit_executions
    circuit_executions += 1
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]


In [26]:
params = random_params(num_wires=5, num_layers=6)

In [27]:
kernel_value = kernel(X[0], X[1], params)
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")

The kernel value between the first and second datapoint is 0.314


In [28]:
f_kernel = lambda x1, x2: kernel(x1, x2, params)
get_kernel_matrix = lambda x1, x2: qml.kernels.kernel_matrix(x1, x2, f_kernel)
class_1_indices = np.where(Y == 1)[0]
class_2_indices = np.where(Y == -1)[0]

class1 = X[class_1_indices]
class2 = X[class_2_indices]

class1_kernel_matrix = get_kernel_matrix(class1, class1)
class2_kernel_matrix = get_kernel_matrix(class2, class2)

main_cluster = KMeans(n_clusters=1, random_state=42)
cluster = KMeans(n_clusters=2, random_state=42)

class1_cent_cluster = main_cluster.fit_predict(class1_kernel_matrix)
centroid1 = main_cluster.cluster_centers_
centroid1 = centroid1[:, :len(X[0])]

class2_cent_cluster = main_cluster.fit_predict(class2_kernel_matrix)
centroid2 = main_cluster.cluster_centers_
centroid2 = centroid1[:, :len(X[0])]

class1_clusters = cluster.fit_predict(class1_kernel_matrix)
class1_centroids = cluster.cluster_centers_
class1_centroids = class1_centroids[:, :len(X[0])]
class1_clusters = np.where(class1_clusters == 1, -1, 1)


class2_clusters = cluster.fit_predict(class2_kernel_matrix)
class2_centroids = cluster.cluster_centers_
class2_centroids = class2_centroids[:, :len(X[0])]
class2_clusters = np.where(class2_clusters == 1, -1, 1)


centroid1_labels = np.ones(len(centroid1))  # Assign all labels as 1 for centroid1
centroid2_labels = -np.ones(len(centroid2))  # Assign all labels as -1 for centroid2

# Concatenate the labels for both centroids
centroid_labels = np.concatenate([centroid1_labels, centroid2_labels])

In [ ]:
class_1_indices = np.where(Y == 1)[0]
class_2_indices = np.where(Y == -1)[0]

class1 = X[class_1_indices]
class2 = X[class_2_indices]

centroid1 = np.average(class1)
centroid2 = np.average(class2)

class1_centroids = []
class2_centroids = []

class1_centroids.append(np.average(np.where(class1 < centroid1)))
class2_centroids.append(np.average(np.where(class2 < centroid1)))

class1_centroids.append(np.average(np.where(class1 > centroid1)))
class2_centroids.append(np.average(np.where(class2 > centroid1)))

centroid1_labels = np.ones(len(centroid1))  # Assign all labels as 1 for centroid1
centroid2_labels = -np.ones(len(centroid2))  # Assign all labels as -1 for centroid2

# Concatenate the labels for both centroids
centroid_labels = np.concatenate([centroid1_labels, centroid2_labels])

In [29]:
def centroid_kernel_matrix(X, centroid, kernel):
    
    kernel_matrix = []

    for i in range(len(X)):
        kernel_matrix.append(kernel(centroid, X[i]))

    return np.array(kernel_matrix)

In [30]:
f_kernel = lambda x1, x2: kernel(x1, x2, params)[0]
centroid_kernel_matrix(class1_centroids, centroid1, f_kernel)

tensor([0.9313973 , 0.48968668], requires_grad=True)

In [31]:

def centroid_target_alignment(X, Y, centroid, kernel, assume_normalized_kernel=False, rescale_class_labels=True):
    
    K = centroid_kernel_matrix(X, centroid, kernel)
    T = np.outer(Y, Y)
    numerator = np.sum(Y * K)  
    denominator = np.sqrt(np.sum(K * K) * np.sum(Y * Y))

    TA = numerator / denominator

    return TA

In [32]:

get_kernel_matrix = lambda x1, x2: qml.kernels.kernel_matrix(x1, x2, f_kernel)
km = centroid_target_alignment(class1_centroids, np.array([1, 1]), centroid1, f_kernel)
km

tensor(0.95493383, requires_grad=True)

In [33]:
def loss(X, Y, centroid, kernel, params, lambda_kao = 0.01):
    TA = centroid_target_alignment(X, Y, centroid, kernel)
    r = lambda_kao * np.sum(params ** 2)
    return 1 - TA + r

In [34]:
f_kernel = lambda x1, x2: kernel(x1, x2, params)[0]
loss(class1_centroids, np.array([1, 1], requires_grad=False), centroid1, f_kernel, params)

tensor(9.16186629, requires_grad=True)

In [35]:
main_centroid = True
kao_class = 1
opt = qml.GradientDescentOptimizer(0.2)
circuit_executions = 0
params = random_params(num_wires=5, num_layers=6)
for i in range(500):
    
    if main_centroid:
        if kao_class == 1:
            cost = lambda _params: loss(
                                
                                        class1_centroids, 
                                        centroid1_labels,
                                        centroid1,
                                        lambda x1, x2: kernel(x1, x2, params)[0],
                                        _params
                                        )
            kao_class = 2
        else:

            cost = lambda _params: loss(
                                
                                        class2_centroids, 
                                        centroid2_labels,
                                        centroid2,
                                        lambda x1, x2: kernel(x1, x2, params)[0],
                                        _params
                                        )
            kao_class = 1
            main_centroid = False

    else:

    #    cost = lambda _params: -qml.kernels.target_alignment(
    #                                                            class1_centroids + class2_centroids,
    #                                                            centroid1_labels,
    #                                                            lambda x1, x2: kernel(x1, x2, _params),
    #                                                            assume_normalized_kernel=True,
    #                                                        )
        kao_class = 1
        main_centroid = True

    print(params)
    print(cost(params), main_centroid, kao_class)
    params = opt.step(cost, params)
    
    
    if (i + 1) % 50 == 0:
        current_alignment = qml.kernels.target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")
        print(f"Circuit Executions: {circuit_executions}")  


[[[3.36263051e+00 4.23476339e+00 3.10213417e+00 2.09436916e+00
   4.71923191e+00]
  [1.37476254e+00 5.23575726e+00 4.25435597e-01 3.75376715e+00
   1.52001105e+00]]

 [[5.55817595e+00 1.30949688e+00 3.55089511e+00 1.66111764e+00
   3.00877517e+00]
  [1.36854748e+00 4.17593285e+00 5.78750075e+00 3.59730820e+00
   4.07221882e+00]]

 [[4.91006835e+00 5.96340740e+00 2.10766780e-01 4.14124136e+00
   3.99213590e-01]
  [1.44966280e+00 3.74597964e+00 5.39157995e+00 2.85981936e+00
   2.20868847e+00]]

 [[1.19566020e-01 5.33600571e+00 9.57876996e-01 4.07449200e+00
   3.41639424e+00]
  [5.64783103e+00 4.82773422e+00 2.90007534e+00 5.21217040e+00
   2.73275908e+00]]

 [[2.39489282e+00 3.33744187e+00 5.36828639e+00 3.91118544e+00
   2.35840833e+00]
  [5.56713130e+00 6.12443123e+00 1.19904457e+00 1.95756365e+00
   8.58230456e-01]]

 [[1.02107635e+00 5.90433591e+00 1.54622511e+00 2.66270338e-03
   5.89343432e+00]
  [3.84554629e+00 1.62952586e+00 3.43071804e+00 2.51403870e+00
   6.23778566e+00]]]
7.93

'\n        f_kernel = lambda x1, x2: kernel(x1, x2, params)\n        get_kernel_matrix = lambda x1, x2: qml.kernels.kernel_matrix(x1, x2, f_kernel)\n        class_1_indices = np.where(Y == 1)[0]\n        class_2_indices = np.where(Y == -1)[0]\n\n        class1 = X[class_1_indices]\n        class2 = X[class_2_indices]\n\n        class1_kernel_matrix = get_kernel_matrix(class1, class1)\n        class2_kernel_matrix = get_kernel_matrix(class2, class2)\n\n        main_cluster = KMeans(n_clusters=1, random_state=42)\n        cluster = KMeans(n_clusters=4, random_state=42)\n\n        class1_cent_cluster = main_cluster.fit_predict(class1_kernel_matrix)\n        centroid1 = main_cluster.cluster_centers_\n        centroid1 = centroid1[:, :len(X[0])]\n\n        class2_cent_cluster = main_cluster.fit_predict(class2_kernel_matrix)\n        centroid2 = main_cluster.cluster_centers_\n        centroid2 = centroid1[:, :len(X[0])]\n\n\n        class1_clusters = cluster.fit_predict(class1_kernel_matrix)

In [36]:
from sklearn.svm import SVC
# First create a kernel with the trained parameter baked into it.
trained_kernel = lambda x1, x2: kernel(x1, x2, params)

# Second create a kernel matrix function using the trained kernel.
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)

# Note that SVC expects the kernel argument to be a kernel matrix function.
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

In [37]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [38]:
accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 1.000


In [39]:
accuracy_trained = accuracy(svm_trained, x_test, y_test)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 1.000


In [42]:
circuit_executions = 0

params = random_params(num_wires=5, num_layers=6)
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    subset = np.random.choice(list(range(len(X))), 4)
    # Define the cost function for optimization
    cost = lambda _params: -qml.kernels.target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 50 == 0:
        current_alignment = qml.kernels.target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

Step 50 - Alignment = 0.438
Step 100 - Alignment = 0.444
Step 150 - Alignment = 0.448
Step 200 - Alignment = 0.452
Step 250 - Alignment = 0.455
Step 300 - Alignment = 0.457
Step 350 - Alignment = 0.459
Step 400 - Alignment = 0.462
Step 450 - Alignment = 0.464
Step 500 - Alignment = 0.465


In [43]:
from sklearn.svm import SVC
# First create a kernel with the trained parameter baked into it.
trained_kernel = lambda x1, x2: kernel(x1, x2, params)

# Second create a kernel matrix function using the trained kernel.
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)

# Note that SVC expects the kernel argument to be a kernel matrix function.
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

In [44]:
accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 1.000


In [45]:
accuracy_trained = accuracy(svm_trained, x_test, y_test)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 1.000


In [46]:
circuit_executions

77970